In [1]:
import json
import torch
from sentence_transformers import SentenceTransformer

In [14]:
def generate_prompt(data, task):
    # sorry about the formatting disaster gotta move fast
    if task == "SeqRec":         # we embed the whole data (input + output) in this case
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  

### Instruction:
{data["instruction"]}

### Input:
{data["input"]}

### Response:
{data["output"]}
"""
    elif task == "CTRPre":       # we embed the input in this case  
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data["instruction"]}

### Input:
{data["input"]}

### Response:
"""
        
def generate_embeddings(data_path, model_path, task):
    model = SentenceTransformer(model_path)
    with open(data_path, "r") as f:
        data = json.load(f)
    prompts = [generate_prompt(item, task) for item in data]
    embeddings = model.encode(prompts, show_progress_bar=True)
    return embeddings

### SeqRec

In [9]:
dataset = "games"
task = "SeqRec"
encoder_path = "../../models/Encoders/Roberta-base"

In [ ]:
for split in ["train", "valid"]:
    embeddings = generate_embeddings(f"../../datasets/{dataset}/{task}/{split}.json", encoder_path, task)
    torch.save(torch.from_numpy(embeddings), f"../../datasets/{dataset}/{task}/{split}_embeddings.pth")

### CTRPre

In [ ]:
task = "CTRPre"
for split in ["train", "valid"]:
    embeddings = generate_embeddings(f"../../datasets/{dataset}/{task}/{split}.json", encoder_path, task)
    torch.save(torch.from_numpy(embeddings), f"../../datasets/{dataset}/{task}/{split}_embeddings.pth")